In [1]:
import pandas as pd
import numpy as np
from src.cleaner import *
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
pd.set_option('max_columns', 1000)

In [2]:
user=pd.read_csv('data/user_reviews_top2600.csv')

In [111]:
items=pd.read_csv('data/first_mens_scrape8660.csv')

In [112]:
df=cleaner(items)
df=rating_unpack(df)
df=create_columns_from_dic(df,'main accords')
df=notes_unpack(df)
df=df.drop(304)
items=df.drop(columns=['_id','image','description','top notes id','mid notes id','base notes id','user voted notes','user review date','similiar fragrences by user vote'])

In [113]:
df=items

In [114]:
#normalize data of user voted columns
sillage=df[['close to skin','radiates about arm length','radiates 6ft','fills room']]
sig=df[['have it','had it','want it','signature']]
season=df[['winter','spring','summer','autumn']]
tod=df[['day','night']]
ratings=df[['female love u25','female love o25','male love u25','male love o25','female like u25','female like o25','male like u25','male like o25','female dislike u25','female dislike o25','male dislike u25','male dislike o25']]
normalized_sillage = preprocessing.normalize(sillage,axis=0)
normalized_sig = preprocessing.normalize(sig,axis=0)
normalized_season = preprocessing.normalize(season,axis=0)
normalized_tod = preprocessing.normalize(tod,axis=0)
normalized_ratings = preprocessing.normalize(ratings,axis=0)

In [115]:
#create sillage df with normalized votes
df['stay on skin']=normalized_sillage[:,0]
df['arm length']=normalized_sillage[:,1]
df['radiate 6ft']=normalized_sillage[:,2]
df['full room']=normalized_sillage[:,3]

#add ratings with normalized votes to df
df['female love u25']=normalized_ratings[:,0]
df['female love o25']=normalized_ratings[:,1]
df['male love u25']=normalized_ratings[:,2]
df['male love o25']=normalized_ratings[:,3]
df['female like u25']=normalized_ratings[:,4]
df['female like o25']=normalized_ratings[:,5]
df['male like u25']=normalized_ratings[:,6]
df['male like o25']=normalized_ratings[:,7]
df['female dislike u25']=normalized_ratings[:,8]
df['female dislike o25']=normalized_ratings[:,9]
df['male dislike u25']=normalized_ratings[:,10]
df['male dislike o25']=normalized_ratings[:,11]

df['day']=normalized_tod[:,0]
df['night']=normalized_tod[:,1]

df['winter']=normalized_season[:,0]
df['spring']=normalized_season[:,1]
df['summer']=normalized_season[:,2]
df['autumn']=normalized_season[:,3]

df['have it']=normalized_sig[:,0]
df['had it']=normalized_sig[:,1]
df['want it']=normalized_sig[:,2]
df['signature']=normalized_sig[:,3]

In [116]:
time=df[['30min to 1hr','1hr to 2hr','3hr to 6hr','7hr to 12hr','greater than 12hr']]
normalized_time=preprocessing.normalize(time,axis=0)

df['30min to 1hr']=normalized_time[:,0]
df['1hr to 2hr']=normalized_time[:,1]
df['3hr to 6hr']=normalized_time[:,2]
df['7hr to 12hr']=normalized_time[:,3]
df['greater than 12hr']=normalized_time[:,4]

In [117]:
df.drop(columns=['close to skin','radiates about arm length','radiates 6ft','fills room','user reviews','price'],inplace=True)

In [118]:
df=df[df['avg rating']>=4.01]

In [119]:
df.drop(columns='avg rating',inplace=True)

In [11]:
user.rename(columns={'Unnamed: 0':'User ID'},inplace=True)

In [165]:
#accords=df.iloc[:,28:113]

In [120]:
normalized_accords = preprocessing.normalize(accords,axis=0)

In [121]:
for idx,i in enumerate(accords.columns):
    df[i]=normalized_accords[:,idx]

In [57]:
colognes=user.iloc[:,1:]

In [59]:
user1=pd.melt(user,id_vars=['User ID'],value_vars=colognes)

In [62]:
user=user1[user1['value']!=0]

In [65]:
user[user['User ID']==8]

,User ID,variable,value
8,8,Insurrection II Wild Reyane Tradition for men,5.0
85124,8,New York for Gentlemen Brooks Brothers for men,3.0
106403,8,Zara W/END till 3:00 AM Zara for men,5.0
361751,8,Modern Man Banana Republic for men,5.0
510704,8,D Collection Zara for men,4.0
...,...,...,...
53240066,8,Bowling Green Geoffrey Beene for men,5.0
53282624,8,Homme Carven for men,5.0
53303903,8,L'Homme Lacoste Lacoste Fragrances for men,4.0
53495414,8,Dunhill for Men Alfred Dunhill for men,3.0


### Surprise

In [18]:
import surprise
from surprise import Reader, Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate 
from surprise.model_selection import train_test_split
from surprise.model_selection import RandomizedSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import SVD, NMF, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [16]:
reader = Reader(rating_scale=(1, 5))

In [64]:
data = Dataset.load_from_df(user[['User ID','variable','value']], reader)

In [66]:
trainset, testset = train_test_split(data)

### NMF

In [67]:
nmf=NMF()
nmf.fit(trainset)

In [68]:
nmf_predict=nmf.test(testset)

In [70]:
accuracy.rmse(nmf_predict)

RMSE: 0.9970


0.996976417561509

### Baseline (ALS)

In [72]:
baseline=BaselineOnly()
baseline.fit(trainset)

Estimating biases using als...


In [73]:
baseline_predict=baseline.test(testset)
accuracy.rmse(baseline_predict)

RMSE: 0.8716


0.8716127484958387

### SVD

In [74]:
svd=SVD()
svd.fit(trainset)

In [75]:
svd_predict=svd.test(testset)
accuracy.rmse(svd_predict)

RMSE: 0.8807


0.880732047212817

### KNN

In [76]:
knnb=KNNBasic()
knnb.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [78]:
knnb_predict=knnb.test(testset)
accuracy.rmse(knnb_predict)

RMSE: 0.9168


0.9167521675594821

### KNN Baseline

In [79]:
knnbase=KNNBaseline()
knnbase.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [80]:
knnbase_predict=knnbase.test(testset)
accuracy.rmse(knnbase_predict)

RMSE: 0.8974


0.8974013360034244

### Item Recommender


In [81]:
from src.rec import *

In [123]:
item_name=df.pop('name')

In [101]:
#item_name=list(item_name)

In [189]:
rec=ItemRecommender()

In [159]:
rec.fit(df)

In [190]:
item_name[item_name.str.contains('Chanel')]

2309                         Pour Monsieur Chanel for men
2620            Allure Homme Sport Cologne Chanel for men
3592          Allure Homme Edition Blanche Chanel for men
4210              Pour Monsieur Concentree Chanel for men
4617                     Allure Pour Homme Chanel for men
4642    Allure Homme Edition Blanche Eau de Parfum Cha...
4856    Allure Homme Eau Fraichissante Pour l'Ete Chan...
5771                               Antaeus Chanel for men
5787           Pour Monsieur Eau de Parfum Chanel for men
5948                      Egoiste Platinum Chanel for men
6660                        Bleu de Chanel Chanel for men
6661        Allure Homme Sport Eau Extreme Chanel for men
6662          Bleu de Chanel Eau de Parfum Chanel for men
6691                 Bleu de Chanel Parfum Chanel for men
7237                    Allure Homme Sport Chanel for men
7243                               Egoiste Chanel for men
7340            Egoiste Cologne Concentree Chanel for men
Name: name, dt

In [173]:
rec.similarity_df

0         1         2         3         4         5         6     \
5     1.000000  0.019422  0.006883  0.006024  0.009387  0.011812  0.004895   
6     0.019422  1.000000  0.412167  0.133728  0.227966  0.168404  0.436139   
8     0.006883  0.412167  1.000000  0.308496  0.263262  0.291372  0.315064   
13    0.006024  0.133728  0.308496  1.000000  0.319522  0.157405  0.204089   
23    0.009387  0.227966  0.263262  0.319522  1.000000  0.134502  0.348298   
...        ...       ...       ...       ...       ...       ...       ...   
7476  0.006496  0.308984  0.089221  0.000270  0.246248  0.000350  0.117990   
7477  0.007858  0.142826  0.164846  0.000233  0.000314  0.338043  0.109112   
7479  0.003346  0.545063  0.378027  0.204075  0.261544  0.385331  0.333537   
7480  0.007521  0.169990  0.097617  0.000290  0.134864  0.000547  0.129212   
7481  0.006328  0.377851  0.218299  0.235585  0.201136  0.000423  0.192577   

          7         8         9         10        11        12        13    \
5     0.005183  0.007668  0.007504  0.009277  0.006812  0.009816  0.007350   
6     0.327102  0.000372  0.308301  0.000359  0.425949  0.000499  0.169410   
8     0.189174  0.165189  0.267438  0.088932  0.328104  0.000236  0.292770   
13    0.204560  0.134263  0.000455  0.145839  0.266210  0.000354  0.315892   
23    0.174321  0.000391  0.123355  0.000317  0.113523  0.000338  0.135096   
...        ...       ...       ...       ...       ...       ...       ...   
7476  0.117981  0.308592  0.166794  0.000335  0.153598  0.000429  0.000345   
7477  0.000231  0.428154  0.154210  0.155341  0.141996  0.001332  0.000444   
7479  0.250129  0.000342  0.118113  0.000210  0.325421  0.000293  0.387278   
7480  0.000349  0.168982  0.000544  0.003408  0.168329  0.000360  0.000683   
7481  0.192540  0.000357  0.136427  0.135715  0.375663  0.000472  0.149382   

          14        15        16        17        18        19        20    \
5     0.005140  0.005697  0.006395  0.005228  0.009815  0.005336  0.005500   
6     0.238887  0.000593  0.000384  0.227696  0.000505  0.455527  0.000402   
8     0.413767  0.195367  0.109394  0.131579  0.000211  0.394878  0.089385   
13    0.224604  0.316727  0.176636  0.000219  0.000337  0.319951  0.145036   
23    0.190696  0.269814  0.151023  0.181800  0.000322  0.091243  0.123365   
...        ...       ...       ...       ...       ...       ...       ...   
7476  0.129193  0.182748  0.000238  0.246012  0.000522  0.000240  0.000318   
7477  0.000261  0.168818  0.188906  0.114631  0.001324  0.227725  0.000590   
7479  0.273751  0.129422  0.000318  0.174041  0.000243  0.348276  0.000349   
7480  0.141251  0.199829  0.001278  0.134597  0.000440  0.000327  0.000477   
7481  0.316105  0.000522  0.333153  0.402068  0.000399  0.201141  0.000331   

          21        22        23        24        25        26        27    \
5     0.005672  0.006100  0.006891  0.008983  0.005020  0.007035  0.008930   
6     0.267207  0.425933  0.000600  0.308776  0.308447  0.126120  0.377428   
8     0.308685  0.328125  0.082694  0.089243  0.356346  0.145830  0.327015   
13    0.125078  0.266188  0.134004  0.000919  0.144396  0.117879  0.000341   
23    0.213468  0.113518  0.114263  0.123147  0.246359  0.000406  0.150876   
...        ...       ...       ...       ...       ...       ...       ...   
7476  0.000203  0.153552  0.154445  0.166740  0.166782  0.000265  0.203940   
7477  0.133601  0.141979  0.143184  0.001426  0.154210  0.000289  0.377312   
7479  0.408369  0.325449  0.109337  0.353168  0.471540  0.096428  0.288597   
7480  0.000376  0.168280  0.169374  0.365213  0.182865  0.000319  0.223472   
7481  0.118046  0.375697  0.126135  0.136161  0.136268  0.222319  0.166831   

          28        29        30        31        32        33        34    \
5     0.004338  0.004086  0.004029  0.007493  0.004854  0.003223  0.005077   
6     0.266756  0.126215  0.105140  0.189502  0.101258  0.238999  0.143033   
8     0.385215  0.218319  0.

In [149]:
rec.get_recommendations(6661)

array([7249, 3263,  971, 2181, 4118])

In [156]:
item_name[7249]

'Chic For Men Carolina Herrera for men'

In [163]:
duration=df[['30min to 1hr','1hr to 2hr','3hr to 6hr','7hr to 12hr','greater than 12hr']]
toy=df[['winter','spring','summer','autumn']]
signature=df['signature']
tod=df[['day','night']]
projection_=df[['stay on skin','arm length','radiate 6ft','full room']]
females=df[['female love u25','female love o25']]

In [167]:
from sklearn.metrics.pairwise import cosine_similarity

In [199]:
cos_duration=cosine_similarity(duration)
cos_toy=cosine_similarity(toy)
cos_df=cosine_similarity(df)
cos_females=cosine_similarity(females)

In [178]:
cos_toy

array([[1.        , 0.70385745, 0.43821348, ..., 0.68252657, 0.71590868,
        0.9788325 ],
       [0.70385745, 1.        , 0.94203744, ..., 0.98032509, 0.95788751,
        0.78384786],
       [0.43821348, 0.94203744, 1.        , ..., 0.94429722, 0.85660502,
        0.53568874],
       ...,
       [0.68252657, 0.98032509, 0.94429722, ..., 1.        , 0.91661743,
        0.73174509],
       [0.71590868, 0.95788751, 0.85660502, ..., 0.91661743, 1.        ,
        0.7984052 ],
       [0.9788325 , 0.78384786, 0.53568874, ..., 0.73174509, 0.7984052 ,
        1.        ]])

In [179]:
def weighted_vector(matrix1, matrix2, alpha):
    return (matrix1 * alpha) + (matrix2 * (1-alpha))

In [200]:
weight=weighted_vector(cos_df,cos_females,.2)

In [201]:
rec.fit(weight,index=item_name)

In [202]:
rec.similarity_df

0         1     \
name                                                                     
Insurrection II Wild Reyane Tradition for men       1.000000  0.991652   
White Musk For Men The Body Shop for men            0.991652  1.000000   
New West for Him Aramis for men                     0.987068  0.961264   
Homme Nature Yves Rocher for men                    0.999445  0.991811   
New York for Gentlemen Brooks Brothers for men      0.998386  0.985639   
...                                                      ...       ...   
Twilight Woods for Men Bath and Body Works for men  0.935573  0.971257   
Burberry London for Men Special Edition 2009 Bu...  0.996660  0.979769   
Vetiver L.T. Piver for men                          0.997160  0.997825   
212 Men Ice Carolina Herrera for men                0.987146  0.998531   
Fever pour Homme Celine for men                     0.996001  0.978932   

                                                        2         3     \
name                                                                     
Insurrection II Wild Reyane Tradition for men       0.987068  0.999445   
White Musk For Men The Body Shop for men            0.961264  0.991811   
New West for Him Aramis for men                     1.000000  0.987543   
Homme Nature Yves Rocher for men                    0.987543  1.000000   
New York for Gentlemen Brooks Brothers for men      0.993181  0.998581   
...                                                      ...       ...   
Twilight Woods for Men Bath and Body Works for men  0.869548  0.935151   
Burberry London for Men Special Edition 2009 Bu...  0.996110  0.996464   
Vetiver L.T. Piver for men                          0.975924  0.997450   
212 Men Ice Carolina Herrera for men                0.950611  0.986762   
Fever pour Homme Celine for men                     0.996516  0.996125   

                                                        4         5     \
name                                                                     
Insurrection II Wild Reyane Tradition for men       0.998386  0.997183   
White Musk For Men The Body Shop for men            0.985639  0.997502   
New West for Him Aramis for men                     0.993181  0.975360   
Homme Nature Yves Rocher for men                    0.998581  0.997310   
New York for Gentlemen Brooks Brothers for men      1.000000  0.993419   
...                                                      ...       ...   
Twilight Woods for Men Bath and Body Works for men  0.919394  0.955813   
Burberry London for Men Special Edition 2009 Bu...  0.998655  0.989596   
Vetiver L.T. Piver for men                          0.993834  0.999541   
212 Men Ice Carolina Herrera for men                0.979194  0.994734   
Fever pour Homme Celine for men                     0.998692  0.988377   

                                                        6         7     \
name                                                                     
Insurrection II Wild Reyane Tradition for men       0.986403  0.994456   
White Musk For Men The Body Shop for men            0.960142  0.974877   
New West for Him Aramis for men                     0.999468  0.997627   
Homme Nature Yves Rocher for men                    0.986732  0.994531   
New York for Gentlemen Brooks Brothers for men      0.992749  0.997784   
...                                                      ...       ...   
Twilight Woods for Men Bath and Body Works for men  0.867477  0.895857   
Burberry London for Men Special Edition 2009 Bu...  0.995621  0.998903   
Vetiver L.T. Piver for men                          0.974916  0.986184   
212 Men Ice Carolina Herrera for men                0.949337  0.966321   
Fever pour Homme Celine for men                     0.996106  0.999187   

                                                        8         9     \
name                                                                     
Insurrection II Wild Reyane Tradition for men       0.99

In [203]:
rec.get_recommendations('Allure Homme Sport Eau Extreme Chanel for men')

array(["Lanvin L'Homme Lanvin for men",
       'Burberry Summer for Men 2013 Burberry for men',
       'Moschino Forever Moschino for men',
       'Metropolis Estée Lauder for men',
       'Herod Parfums de Marly for men'], dtype=object)